In [1]:
!pip install -q langchain langchain-community langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.4/412.4 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.4 MB/s eta 0:00:00


In [2]:
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.base import RunnableSequence
import re
from langchain_groq import ChatGroq
import os
import getpass
import time

In [3]:
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your GROQ API key: ")

Enter your GROQ API key: ··········


In [14]:
def clean_text(text):
  #Remove special characters and normalize whitespace
  cleaned_text = re.sub(r'[^A-Za-z0-9\s]', '', text)
  return re.sub(r'\s+', ' ', cleaned_text).strip()

def transform_text(cleaned_text):
  #Convert text to uppercase for standardization
  return cleaned_text.upper()

def gemini_sentiment_analysis(text):

  chat_model = ChatGroq(model_name="gemma2-9b-it")

  prompt = f"Analyze the sentiment of the following text and classify it as positive, negative, or neutral:\n{text}"

  response = chat_model.invoke(prompt)
  sentiment = response.content.strip().lower()

  return {"text": text, "sentiment": sentiment}

def aggregate_sentiments(sentiment_results):
  summary = {'positive': 0, 'negative': 0, 'neutral': 0}
  for result in sentiment_results:
    if result['sentiment'] in summary:
      summary[result['sentiment']] += 1

  return summary

In [15]:
runnable_clean = RunnableLambda(lambda texts: [clean_text(text) for text in texts])
runnable_transform = RunnableLambda(lambda cleaned_texts: [transform_text(text) for text in cleaned_texts])
runnable_sentiment = RunnableLambda(lambda transformed_texts: [gemini_sentiment_analysis(text) for text in transformed_texts])
# runnable_aggregate = RunnableLambda(aggregate_sentiments)

In [16]:
review_analysis_pipeline = RunnableSequence(
    first = runnable_clean,
    middle = [runnable_transform],
    last = runnable_sentiment
)

In [17]:
reviews = [
    "The product quality is excellent and the service was awesome!",
    "Very poor experience, the product was bad and delivery was horrible.",
    "Good value for money, but could be better.",
    "I had a bad experience, but the customer support was good.",
    "This is the best product I've ever bought!"
]

In [19]:
result = review_analysis_pipeline.invoke(reviews)
print(result)

[{'text': 'THE PRODUCT QUALITY IS EXCELLENT AND THE SERVICE WAS AWESOME', 'sentiment': 'the sentiment of the text is **strongly positive**. \n\nhere\'s why:\n\n* **"excellent"** is a highly positive adjective.\n* **"awesome"** is a very strong positive term.\n\nthe use of these words clearly expresses a high level of satisfaction with both the product and the service. \n\n\nlet me know if you have any other text you\'d like me to analyze!'}, {'text': 'VERY POOR EXPERIENCE THE PRODUCT WAS BAD AND DELIVERY WAS HORRIBLE', 'sentiment': 'the sentiment of the text is **negative**. \n\nhere\'s why:\n\n* **strong negative words:**  "very poor experience", "bad", and "horrible" all express strong dissatisfaction and negativity.\n* **lack of positive sentiment:** there are no words or phrases that convey any positive feelings or experiences.\n\n\nthe text clearly expresses a very negative opinion about both the product and the delivery service.'}, {'text': 'GOOD VALUE FOR MONEY BUT COULD BE BETT